Transfer Learning and TensorFlow 2.0 to Classify Different Dog Breeds




In this project we're going to be using deep learning to help us identify different breeds of dogs.

To do this, we'll be using data from the [Kaggle dog breed identification competition](https://www.kaggle.com/c/dog-breed-identification/overview).

It is an example of multi class classification.

Since the most important step in a deep learng problem is getting the data ready (turning it into numbers), that's what we're going to start with.

We're going to go through the following TensorFlow/Deep Learning workflow:
1. Get data ready (download from Kaggle, store, import).
2. Prepare the data (preprocessing, the 3 sets, X & y).
3. Choosing a model.
4. Evaluating a model.
5. Improve the model through experimentation (start with 1000 images, make sure it works, increase the number of images).


In [ ]:
# import and check tensorflow
import tensorflow as tf
print("TF version:", tf.__version__)

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)

# Check for GPU
if tf.config.list_physical_devices("GPU"):
    print("GPU is available")
else:
    print("not available")



### Accessing the data

Import the labels.csv file

In [ ]:

import pandas as pd
labels = pd.read_csv("../input/dog-breed-identification/labels.csv")
print(labels.head())

Looking at this, we can see there are 10222 different ID's (meaning 10222 different images) and 120 different breeds.

Let's figure out how many images there are of each breed.

In [ ]:
#columns of the labels.csv file
labels.columns

In [ ]:
#displaying an image from the data
from IPython.display import display, Image
Image("../input/dog-breed-identification/train/001cdf01b096e06d78e9e5112d419397.jpg")

### Images and labels

Creating list of images from their label IDs 

In [ ]:
# Creating pathnames
filenames = []
for name in labels["id"]:
    filenames.append("../input/dog-breed-identification/train/" + name + ".jpg")

# Check the first 10 filenames
filenames[:10]

Check if all files are complete

In [ ]:
# Check whether number of filenames matches number of actual image files
import os
if len(os.listdir("../input/dog-breed-identification/train/")) == len(filenames):
  print("There are no missing files.")
else:
  print("Files missing")

Visualizing one of the images from the created list

In [ ]:
# Check an image directly from a filepath
Image(filenames[60])

Cute!

Now we've got our image filepaths together, let's get the labels.

We'll take them from `labels_csv` and turn them into a NumPy array.

In [ ]:
import numpy as np
label = labels["breed"].to_numpy() # convert labels column to NumPy array
label[:10]

Wonderful, now lets do the same thing as before, compare the amount of labels to number of filenames.

In [ ]:
# See if number of labels matches the number of filenames
if len(label) == len(filenames):
  print("Exact number of files")
else:
  print("Some files are missing")

In [ ]:
# Find the number of unique label
unique = np.unique(label)
len(unique)

Converting array of numbers to boolean array

That's for one example, let's do the whole thing.

In [ ]:
# Turn every label into a boolean array
boolean_label = [element == np.array(unique) for element in label]
boolean_label[:1]

In [ ]:
print(label[0]) # original label
print(np.where(unique == label[0])[0][0]) # index where label occurs in the number array
print(boolean_label[0].argmax()) # index where label occurs in boolean array
print(boolean_label[0].astype(int))

### Creating our own validation set

Using scikit learn train_test_split to split the data into train and validation set.

In [ ]:
# Setup X & y variables
x = filenames
y = boolean_label

Starting with 1000 images

In [ ]:
# Set number of images to use
NUM_IMAGES = 1000


Now let's split our data into training and validation sets. We'll use and 80/20 split (80% training data, 20% validation data).

In [ ]:
# Import train_test_split from Scikit-Learn
from sklearn.model_selection import train_test_split

# Split them into training and validation using NUM_IMAGES 
x_train, x_val, y_train, y_val = train_test_split(x[:NUM_IMAGES],
                                                  y[:NUM_IMAGES], 
                                                  test_size=0.2,
                                                  random_state=42)

len(x_train), len(y_train), len(x_val), len(y_val)

### Turning images into Tensors

Define a function that:
1. Takes an image filename as input.
2. Uses TensorFlow to read the file and save it to a variable, `image`.
3. Turn our `image` into Tensors.
4. Resize the `image` to be of shape (224, 224).
5. Return the modified `image`.

In [ ]:
# Convert image to NumPy array
from matplotlib.pyplot import imread
image = imread(filenames[1])
image.shape# read in an image

3 is the height of the image (red,green,blue)

In [ ]:
tf.constant(image)[:1]

In [ ]:
# Define image dimension
IMG_SIZE = 224

def process(image_path):
  
  # Read in image file
  image = tf.io.read_file(image_path)

  # Turn the jpeg image into numerical Tensor with 3 colour channels (Red, Green, Blue)
  image = tf.image.decode_jpeg(image, channels=3)

  # Convert the colour channel values from 0-225 values to 0-1 values
  image = tf.image.convert_image_dtype(image, tf.float32)

  # Resize the image to our desired size (224, 244)
  image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
  return image

### Creating data batches

In deep learning, instead of finding patterns in an entire dataset at the same time, you often find them one batch at a time.
Here we will create batches of size 32.

In [ ]:
# Creating function to return a tuple (image, label)
def image_label(path, label):
 
  image = process(path)
  return image, label

In [ ]:
# defining batch dimensions
BATCH = 32

# Create a function to turn data into batches
def data_batches(x, y=None, batch_size=BATCH , valid_data=False, test_data=False):
    
  # If the data is a test dataset
  if test_data:
    print("Test data batches created")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x))) # only filepaths
    data_batch = data.map(process).batch(BATCH)
    return data_batch
  
  # If the data if a valid dataset
  elif valid_data:
    
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x), # filepaths
                                               tf.constant(y))) # labels
    data_batch = data.map(image_label).batch(BATCH)
    print("Validation data batches created")
    return data_batch

  else:
    # If the data is a training dataset, we shuffle it
    
    # Turn filepaths and labels into Tensors
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x), # filepaths
                                              tf.constant(y))) # labels
    
    # Shuffling pathnames and labels
    data = data.shuffle(buffer_size=len(x))

    # Create (image, label) tuples
    data = data.map(image_label)

    # Turn the data into batches
    data_batch = data.batch(BATCH)
    print("Training data batches created")
  return data_batch

In [ ]:
# Create training and validation data batches
train_data = data_batches(x_train, y_train)
val_data = data_batches(x_val, y_val, valid_data=True)

In [ ]:
# attributes of our data batches
train_data.element_spec, val_data.element_spec

Visualization

In [ ]:
import matplotlib.pyplot as plt

# Create a function for viewing images in a data batch
def show_images(images, labels):
    
  # Setup the figure
  plt.figure(figsize=(20, 20))
    
  # Loop through 25 (for displaying 25 images)
  for i in range(25):
        
    # Create subplots (5 rows, 5 columns)
    ax = plt.subplot(5, 5, i+1)
    # Display an image
    plt.imshow(images[i])
    # Add the image label as the title
    plt.title(unique[labels[i].argmax()])
    # Turn gird lines off
    plt.axis("off")

In [ ]:
# Visualize training images from the training data batch
train_images, train_labels = next(train_data.as_numpy_iterator())
show_images(train_images, train_labels)

Since we have shuffled the train data in the function , everytime we run he function we will get a different set of pictures for the training data

In [ ]:
# Visualize validation images from the validation data batch
val_images, val_labels = next(val_data.as_numpy_iterator())
show_images(val_images, val_labels)

# Creating and training a model

Using a pretrained machine learning model is often referred to as **transfer learning**.


 We will use [mobilenet_v2_130_224](https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4)


In [ ]:
# Setup input shape
INPUT_SHAPE = [None, IMG_SIZE, IMG_SIZE, 3] # batch, height, width, colour channels

# Setup output shape
OUTPUT_SHAPE = len(unique) # number of unique labels

# model URL from TensorFlow Hub
MODEL_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4"

In [ ]:
# Create a function which builds a Keras model
def modelling(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, model_url=MODEL_URL):

  # Setup the model layers
  model = tf.keras.Sequential([
    hub.KerasLayer(MODEL_URL), # Layer 1 (input layer)
    tf.keras.layers.Dense(units=OUTPUT_SHAPE, 
                          activation="softmax") # Layer 2 (output layer)
  ])

  # Compile the model
  model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(), # loss function to be reduced
      optimizer=tf.keras.optimizers.Adam(), # Adam optimizer
      metrics=["accuracy"] 
  )

  # Build the model
  model.build(INPUT_SHAPE) 
  
  return model

In [ ]:
# Create a model and check its details
created_model = modelling()
created_model.summary()

#### Early Stopping Callback

Early stopping helps to maintain the quality of output of our model and if it starts to go down , it stops the model. 



In [ ]:
# Create early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                  patience=3) # stops after 3 rounds of no improvements

In [ ]:
# Number of times our model passes through the data
EPOCHS = 100 

Training model to fit training data and callback

In [ ]:
# Build a function to train and return a trained model
def train_model():
    
  # Create a model
  model = modelling()

  # Fit the model to the data passing it the callbacks we created
  model.fit(x=train_data,
            epochs=EPOCHS,
            validation_data=val_data,
            validation_freq=1, # check validation metrics every epoch
            callbacks=[early_stopping])
  
  return model

In [ ]:
# Fit the model to the data
model = train_model()

The model is clearly overfitting as it performs far better on the training data than on validation data

## Predictions using the trained model


In [ ]:
# Make predictions on the validation data
predictions = model.predict(val_data, verbose=1) # verbose shows us how long there is to go
predictions

In [ ]:
predictions.shape

Making predictions with our model returns an array with a different value for each label.

In this case, making predictions on the validation data (200 images) returns an array (`predictions`) of arrays, each containing 120 different values (one for each unique dog breed).

These different values are the probabilities or the likelihood the model has predicted a certain image being a certain breed of dog. The higher the value, the more likely the model thinks a given image is a specific breed of dog.

Let's see how we'd convert an array of probabilities into an actual label.

In [ ]:
# First prediction
print(predictions[0])
print(f"Max value (probability of prediction): {np.max(predictions[0])}") # the max probability value predicted by the model
print(f"Sum: {np.sum(predictions[0])}")
print(f"Max index: {np.argmax(predictions[0])}") # the index of where the max value in predictions[0] occurs
print(f"Predicted label: {unique[np.argmax(predictions[0])]}") # the predicted label

Predicting dog breed

In [ ]:

def pred_label(prediction_probabilities):

  return unique[np.argmax(prediction_probabilities)]

prediction_label = pred_label(predictions[0])
prediction_label

Wonderful! Now we've got a list of all different predictions our model has made, we'll do the same for the validation images and validation labels.

Remember, the model hasn't trained on the validation data, during the `fit()` function, it only used the validation data to evaluate itself. So we can use the validation images to visually compare our models predictions with the validation labels.

Since our validation data (`val_data`) is in batch form, to get a list of validation images and labels, we'll have to unbatch it (using [`unbatch()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#unbatch)) and then turn it into an iterator using [`as_numpy_iterator()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#as_numpy_iterator).

Let's make a small function to do so.

In [ ]:
# function to unbatch a batched dataset
def unbatchify(data):
  
  images = []
  labels = []

  # Loop through unbatched data
  for image, label in data.unbatch().as_numpy_iterator():
    images.append(image)
    labels.append(unique[np.argmax(label)])
  return images, labels

# Unbatchify the validation data
val_images, val_labels = unbatchify(val_data)
val_images[0], val_labels[0]

Visualizing the predictions

In [ ]:
def plot_pred(prediction_probabilities, labels, images, n=1):
 
  pred_prob, true_label, image = prediction_probabilities[n], labels[n], images[n]
  
  # Get the pred label
  prediction_label = pred_label(pred_prob)
  
  # Plot image & remove ticks
  plt.imshow(image)
  plt.xticks([])
  plt.yticks([])

  # Change the color of the title depending on if the prediction is right or wrong
  if prediction_label == true_label:
    color = "green"
  else:
    color = "red"

  plt.title("{} {:2.0f}% ({})".format(prediction_label,
                                      np.max(pred_prob)*100,
                                      true_label),
                                      color=color)

In [ ]:
# example prediction, original image and truth label
plot_pred(prediction_probabilities=predictions,
          labels=val_labels,
          images=val_images)

In [ ]:
def plot_pred_conf(prediction_probabilities, labels, n=1):
 
  pred_prob, true_label = prediction_probabilities[n], labels[n]

  # Get the predicted label
  prediction_label = pred_label(pred_prob)

  # Find the top 7 prediction confidence indexes
  pred_indexes_7 = pred_prob.argsort()[-7:][::-1]
    
  # Find the top 7 prediction confidence values
  pred_values_7 = pred_prob[pred_indexes_7]
    
  # Find the top 7 prediction labels
  pred_labels_7 = unique[pred_indexes_7]

  # Setup plot
  top_plot = plt.bar(np.arange(len(pred_labels_7)), 
                     pred_values_7, 
                     color="purple")
  plt.xticks(np.arange(len(pred_labels_7)),
             labels=pred_labels_7,
             rotation="vertical")

  # Change color of true label
  if np.isin(true_label, pred_labels_7):
    top_plot[np.argmax(pred_labels_7 == true_label)].set_color("orange")
  else:
    pass

In [ ]:
plot_pred_conf(prediction_probabilities=predictions,
               labels=val_labels,
               n=10)

In [ ]:
# few predictions and their different values
mult = 0
rows = 2
cols = 2
num_images = rows*cols
plt.figure(figsize=(5*1.5*cols, 5*rows))
for i in range(num_images):
  plt.subplot(rows, 2*cols, 2*i+1)
  plot_pred(prediction_probabilities=predictions,
            labels=val_labels,
            images=val_images,
            n=i+mult)
  plt.subplot(rows, 2*cols, 2*i+2)
  plot_pred_conf(prediction_probabilities=predictions,
                labels=val_labels,
                n=i+mult)
plt.tight_layout(h_pad=1.0)
plt.show()

Thankyou